<a href="https://colab.research.google.com/github/ejihoon6065/-/blob/master/5_4_Sentiment_Analysis%20%2B%20CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [63]:
!ls

'5.4.Sentiment Analysis.ipynb'	 log	   preprocessing  'Untitled Folder'
'5.4.Sentiment Analysis.zip'	 log.zip   test.tsv
 data				 model	   train.tsv


In [64]:
cd 5.4.Sentiment Analysis

[Errno 2] No such file or directory: '5.4.Sentiment Analysis'
/content/Sentiment Analysis


In [65]:
pwd

'/content/Sentiment Analysis'

In [66]:
import pandas as pd
from model.LSTM import LSTM_model
from preprocessing.text_preprocessing import Sentences2LemmaWords
from preprocessing.text_preprocessing import GetUnique
from preprocessing.text_preprocessing import setTokenizer
from preprocessing.text_preprocessing import LemmaWords2Seqeunces
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import time
from log.logger import Logger

In [67]:
folder_path = './data/'
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.tsv', sep='\t')

print('train data shape: ', train.shape)
print('test data shape: ', test.shape)
print('----------[train infomation]----------')
print(train.info())
print('----------[test infomation]----------')
print(test.info())

train data shape:  (156060, 4)
test data shape:  (66292, 3)
----------[train infomation]----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB
None
----------[test infomation]----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66292 entries, 0 to 66291
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PhraseId    66292 non-null  int64 
 1   SentenceId  66292 non-null  int64 
 2   Phrase      66292 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.5+ MB
None


In [68]:
train_tokens = Sentences2LemmaWords(train.Phrase)
test_tokens = Sentences2LemmaWords(test.Phrase)

2020-07-14 08:50:00,135[DEBUG|preprocessing, 17] Sentence2LemmaWords: start


212 / 156060 Phrase

/content/Sentiment Analysis/preprocessing/text_preprocessing.py:23: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 23 of the file /content/Sentiment Analysis/preprocessing/text_preprocessing.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  sentence = BeautifulSoup(sentence).get_text()
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


10810 / 156060 Phrase

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'log'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


...

155956 / 156060 Phrase

2020-07-14 08:52:04,229[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 124.09361577033997sec
2020-07-14 08:52:04,292[DEBUG|preprocessing, 17] Sentence2LemmaWords: start


156060 / 156060 Phrase
66030 / 66292 Phrase

2020-07-14 08:52:57,000[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 52.7087025642395sec


66292 / 66292 Phrase


In [69]:
unique_words, maxlen = GetUnique(train_tokens)
print(len(unique_words), maxlen)

2020-07-14 08:52:57,037[DEBUG|preprocessing, 44] GetUnique: start
2020-07-14 08:52:57,189[DEBUG|preprocessing, 56] GetUnique: finish. 계산 시간 0.15117192268371582sec


13745 48


In [70]:
target = train.Sentiment
y_target = to_categorical(target)
num_classes = y_target.shape[1]

In [71]:
x_train, x_valid, y_train, y_valid = train_test_split(
    train_tokens,
    y_target,
    test_size=0.2,
    stratify=y_target)

In [72]:
tokenizer = setTokenizer(x_train, len(unique_words))
x_train = LemmaWords2Seqeunces(x_train, tokenizer, maxlen)
x_valid = LemmaWords2Seqeunces(x_valid, tokenizer, maxlen)
x_test = LemmaWords2Seqeunces(test_tokens, tokenizer, maxlen)

2020-07-14 08:52:58,417[DEBUG|preprocessing, 63] setTokenizer: start
2020-07-14 08:52:59,377[DEBUG|preprocessing, 70] setTokenizer: finish. 계산 시간 0.9602627754211426sec
2020-07-14 08:52:59,380[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 08:53:01,107[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 1.7272887229919434sec
2020-07-14 08:53:01,112[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 08:53:01,462[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 0.3498494625091553sec
2020-07-14 08:53:01,467[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 08:53:02,105[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 0.6381514072418213sec


In [73]:
model = LSTM_model(len(unique_words), maxlen, num_classes)

logger = Logger('main')
start_time = time.time()
logger.debug('fit start')

model.fit(x_train,
          y_train,
          validation_data=(x_valid, y_valid),
          epochs=5,
          batch_size=1024,
          verbose=1)

fit_time = time.time() - start_time
logger.debug('fit end. fit_time {}'.format(fit_time))

2020-07-14 08:53:02,119[DEBUG|LSTM_model_generate, 9] LSTM_model: start
2020-07-14 08:53:02,343[DEBUG|LSTM_model_generate, 20] LSTM_model: finish
2020-07-14 08:53:02,345[DEBUG|main, 5] fit start
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 124848 samples, validate on 31212 samples
Epoch 1/5
124848/124848 [==============================] - 23s 184us/step - loss: 1.0929 - accuracy: 0.5679 - val_loss: 0.9013 - val_accuracy: 0.6344
Epoch 2/5
124848/124848 [==============================] - 22s 179us/step - loss: 0.8091 - accuracy: 0.6702 - val_loss: 0.8409 - val_accuracy: 0.6573
Epoch 3/5
124848/124848 [==============================] - 23s 182us/step - loss: 0.7337 - accuracy: 0.6970 - val_loss: 0.8313 - val_accuracy: 0.6631
Epoch 4/5
124848/124848 [==============================] - 23s 181us/step - loss: 0.6876 - accuracy: 0.7146 - val_loss: 0.8375 - val_accuracy: 0.6608
Epoch 5/5
124848/124848 [==============================] - 23s 183us/step - loss: 0.6524 - accuracy: 0.7260 - val_loss: 0.8424 - val_accuracy: 0.6647


2020-07-14 08:54:56,834[DEBUG|main, 15] fit end. fit_time 114.48963284492493


In [74]:
y_pred = model.predict_classes(x_test)

submission = pd.DataFrame({
    'PhraseId': test.PhraseId,
    'Sentiment': y_pred
})
submission.to_csv(folder_path + 'submission.csv', index=False)

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dropout,Conv1D,GlobalMaxPool1D,Dense
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.models import load_model

In [76]:
embedding_dim=256
batch_size=256

In [77]:
model=Sequential()
model.add(Embedding(len(unique_words),300))
model.add(Dropout(0.3))
model.add(Conv1D(300,3,padding='valid',activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5,activation='softmax'))

es=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=3)
mc=ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
history=model.fit(x_train,y_train,epochs=20,validation_data=(x_valid,y_valid),callbacks=[es,mc])

Epoch 1/20
3902/3902 [==============================] - ETA: 0s - loss: 0.9692 - acc: 0.6085
Epoch 00001: val_acc improved from -inf to 0.64478, saving model to best_model.h5
3902/3902 [==============================] - 213s 55ms/step - loss: 0.9692 - acc: 0.6085 - val_loss: 0.8499 - val_acc: 0.6448
Epoch 2/20
3901/3902 [============================>.] - ETA: 0s - loss: 0.7871 - acc: 0.6773
Epoch 00002: val_acc improved from 0.64478 to 0.66324, saving model to best_model.h5
3902/3902 [==============================] - 210s 54ms/step - loss: 0.7870 - acc: 0.6773 - val_loss: 0.8216 - val_acc: 0.6632
Epoch 3/20
3902/3902 [==============================] - ETA: 0s - loss: 0.7123 - acc: 0.7035
Epoch 00003: val_acc improved from 0.66324 to 0.66763, saving model to best_model.h5
3902/3902 [==============================] - 202s 52ms/step - loss: 0.7123 - acc: 0.7035 - val_loss: 0.8269 - val_acc: 0.6676
Epoch 4/20
3902/3902 [==============================] - ETA: 0s - loss: 0.6641 - acc: 0.720